# Celeba

```bash
rm -rf /tmp/celeba.share
python -m openfed.tools.launch --nproc_per_node 6  --logdir /tmp \
    benchmark/run.py\
        --fed_init_method file:///tmp/celeba.share\
        --task celeba\
        --samples 10\
        --data_root benchmark/datasets/celeba/data
```